In [1]:
import re
from collections import namedtuple

In [2]:
from typing import NamedTuple

In [3]:
import operator as op
from functools import partial

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def __add__(self, other: "Amounts"): return Amounts(*map(op.add,self,other))
    def __sub__(self, other: "Amounts"): return Amounts(*map(op.sub,self,other))
    def __mul__(self, other):
        if isinstance(other, Amounts):
            return Amounts(*map(op.mul,self,other))
        else:
            return Amounts(*map(partial(op.mul,other),self))
            


In [4]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [5]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("test"))
bps

[Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0)),
 Blueprint(ore=Amounts(ore=2, clay=0, obsidian=0, geode=0), clay=Amounts(ore=3, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=8, obsidian=0, geode=0), geode=Amounts(ore=3, clay=0, obsidian=12, geode=0))]

In [6]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]

evaluate(moves,bps[0])

1 Amounts(ore=1, clay=0, obsidian=0, geode=0)
2 Amounts(ore=2, clay=0, obsidian=0, geode=0)
3 Amounts(ore=1, clay=0, obsidian=0, geode=0)
4 Amounts(ore=2, clay=1, obsidian=0, geode=0)
5 Amounts(ore=1, clay=2, obsidian=0, geode=0)
6 Amounts(ore=2, clay=4, obsidian=0, geode=0)
7 Amounts(ore=1, clay=6, obsidian=0, geode=0)
8 Amounts(ore=2, clay=9, obsidian=0, geode=0)
9 Amounts(ore=3, clay=12, obsidian=0, geode=0)
10 Amounts(ore=4, clay=15, obsidian=0, geode=0)
11 Amounts(ore=2, clay=4, obsidian=0, geode=0)
12 Amounts(ore=1, clay=7, obsidian=1, geode=0)
13 Amounts(ore=2, clay=11, obsidian=2, geode=0)
14 Amounts(ore=3, clay=15, obsidian=3, geode=0)
15 Amounts(ore=1, clay=5, obsidian=4, geode=0)
16 Amounts(ore=2, clay=9, obsidian=6, geode=0)
17 Amounts(ore=3, clay=13, obsidian=8, geode=0)
18 Amounts(ore=2, clay=17, obsidian=3, geode=0)
19 Amounts(ore=3, clay=21, obsidian=5, geode=1)
20 Amounts(ore=4, clay=25, obsidian=7, geode=2)
21 Amounts(ore=3, clay=29, obsidian=2, geode=3)
22 Amounts(or

Amounts(ore=6, clay=41, obsidian=8, geode=9)

In [7]:
def minerals_spent_building_robots(robots, bp):
    minerals = Amounts()
    for i in range(4):
        minerals = minerals + bp[i]*robots[i]   
    return minerals

def can_build_robots(robots, minerals, bp):
    minerals = minerals - minerals_spent_building_robots(robots,bp)
    return all(m >= 0 for m in minerals)


In [8]:

def valid_moves(robots, minerals, bp):    
    # just moves of one robot at a time for now
    for i in range(4):
        robot = [0,0,0,0]
        robot[i] = 1
        if can_build_robots(Amounts(*robot), minerals, bp):
            yield robot

list(valid_moves(Amounts(), Amounts(2),bps[0]))

[[0, 1, 0, 0]]

In [68]:
def lower_bound(robots, minerals, bp, time_left):
    return minerals.geode + robots.geode*time_left

def upper_bound_1(robots, minerals, bp, time_left):
    # assuming we can build one geode robot per round until time_left given infinite resources
    # minerals.geode + robots.geode*time_left + (time_left-1) + (time_left-2) + (time_left -2) + ...
    # the series sum(i for i in range(m)) can be simplified to (m-1)*m/2)
    return minerals.geode + robots.geode*time_left + (time_left)*(time_left-1)//2



upper_bound = upper_bound_1    

In [61]:
assert sum(range(10)) == 10*9//2

True

In [71]:
from collections import deque

def solve(bp: BP, t=24):

    best = 0
    # state is a tuple of (robots, minerals, time_left)
    todo = deque([(Amounts(ore=1), Amounts(), t)])
    visited = {}    
    state = None
    it = 0
    while todo:
        it += 1
        robots, minerals, time_left = state = todo.pop()
        
        vs = (robots, minerals) 
        if vs in visited and visited[vs] >= time_left:
            continue
        else:
            visited[vs] = time_left
            
        best = max(best,
                   lower_bound(robots, minerals, bp, time_left))
        
        if (it % 100_000 == 0): print(it, len(todo), len(visited), time_left, best)

        if time_left == 0:
            continue
        
        if upper_bound(robots, minerals, bp, time_left) <= best:
            continue
        
        # add the do nothing move
        todo.append((robots, minerals + robots, time_left - 1))
        
        for move in valid_moves(robots, minerals, bp):
            new_state = (
                robots + move,
                minerals - minerals_spent_building_robots(move, bp) + robots,
                time_left - 1
            )
            todo.append(new_state)

        
    return best

In [72]:
%time solve(bps[1])

100000 27 84227 5 10
300000 27 220137 5 10
400000 22 280626 7 10
500000 24 331817 5 10
600000 19 392459 6 10
700000 19 441147 6 10
900000 14 543047 7 10
1100000 22 677662 5 10
1600000 18 936208 7 10
1800000 18 1059150 8 10
2100000 35 1204505 3 10
2300000 25 1293580 5 10
2900000 22 1500995 5 12
3200000 16 1634331 6 12
3300000 11 1684553 5 12
3400000 19 1726547 8 12
CPU times: user 48.3 s, sys: 192 ms, total: 48.5 s
Wall time: 48.5 s


12